In [2]:
import numpy as np
from scipy.ndimage import shift, rotate
import random
import copy

# -------------------
original_data = [
[
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,1,0,0],
[0,0,1,0,0,1,0,0],
[0,0,0,1,1,0,0,0],
[0,0,1,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,1,0,0,0],
[0,0,1,1,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,1,1,1,1,0,0,0],
[0,0,0,0,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,1,1,1,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,1,1,1,0,0,0,0],
[0,1,1,1,0,0,0,0],
[0,0,1,1,0,0,0,0],
[0,1,1,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,0,0,1,1,1,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,1,1,0],
[0,0,0,0,0,1,1,1],
[0,0,0,0,0,0,0,1],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,1,0,0]
],
[
[0,1,1,1,1,0,0,0],
[0,1,0,0,0,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,1,1,1,1,1,0],
[0,0,0,0,0,0,1,1],
[0,0,0,0,0,0,1,0],
[0,0,0,0,1,1,0,0],
[0,1,1,1,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,1,1,1,1,0,0,0],
[0,1,0,0,1,1,0,0],
[0,0,0,0,1,1,0,0],
[0,0,1,1,1,0,0,0],
[0,1,0,0,0,1,0,0],
[0,1,0,0,1,0,0,0],
[0,0,1,1,1,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,0,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,1,1,0,0,0],
[1,1,1,1,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,0,1,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,1,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,1,1,0,0],
[1,1,1,1,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,0,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,1,1,1,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,1,1,0,0],
[0,0,1,1,0,0,0,0],
[0,0,0,0,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,1,1,1,1,0,0],
[0,0,0,0,0,1,1,0],
[0,0,1,1,1,1,1,0],
[0,0,0,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,1,1,1,1,0,0,0],
[0,0,0,0,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,1,1,0,0,0,0],
[0,1,0,1,0,0,0,0],
[0,0,1,1,1,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,1,1,0,0,0],
[0,1,1,0,0,0,0,0],
[0,0,0,0,0,0,0,0]
],
[
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0],
[0,1,1,0,0,0,0,0],
[0,0,1,0,0,0,0,0],
[0,1,1,0,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,1,1,0,0,0,0,0]
]
]

def shift_matrix(matrix, dx=0, dy=0):
    """ 平移矩陣 """
    new_matrix = [[0]*8 for _ in range(8)]
    for i in range(8):
        for j in range(8):
            ni = i + dy
            nj = j + dx
            if 0 <= ni < 8 and 0 <= nj < 8:
                new_matrix[ni][nj] = matrix[i][j]
    return new_matrix

def add_smart_noise(matrix, base_noise=0.01, edge_noise=0.02):
    """ 改進的智能噪點：只在數字邊緣附近添加輕微噪點 """
    noisy_matrix = copy.deepcopy(matrix)
    digit_pixels = [(i,j) for i in range(8) for j in range(8) if matrix[i][j] == 1]

    if not digit_pixels:
        return noisy_matrix

    for i in range(8):
        for j in range(8):

            min_dist = min(abs(i-x)+abs(j-y) for (x,y) in digit_pixels)

            if min_dist <= 2:
                dynamic_prob = base_noise + edge_noise * (2 - min_dist)/2
                if random.random() < dynamic_prob:
                    noisy_matrix[i][j] ^= 1

    return noisy_matrix

def scale_matrix(matrix, scale_factor=1.0):
    """ 輕微縮放 """
    if scale_factor == 1.0:
        return copy.deepcopy(matrix)

    np_mat = np.array(matrix, dtype=np.float32)
    scaled = np.zeros((8,8))

    for i in range(8):
        for j in range(8):
            src_i = (i - 4) / scale_factor + 4
            src_j = (j - 4) / scale_factor + 4
            if 0 <= src_i < 8 and 0 <= src_j < 8:
                scaled[i][j] = np_mat[int(src_i)][int(src_j)]

    return (scaled > 0.5).astype(int).tolist()

def rotate_matrix(matrix, k=0):
    """ 有限旋轉 (0或180度) """
    if k == 0:
        return copy.deepcopy(matrix)
    return [row[::-1] for row in matrix[::-1]]

# 擴增參數 (調整為更保守的值)
TARGET_COUNT = 1000
MAX_SHIFT = 1          # 保持±1像素位移
BASE_NOISE = 0.01      # 基礎噪點概率 (1%)
EDGE_NOISE = 0.02      # 邊緣額外噪點概率 (2%)
MIN_SCALE = 0.95       # 縮放範圍縮小
MAX_SCALE = 1.05       # 避免過度變形

# 生成增強資料
augmented_data = []
while len(augmented_data) < TARGET_COUNT:
    for mat in original_data:
        # 1. 平移 (最重要)
        dx = random.randint(-MAX_SHIFT, MAX_SHIFT)
        dy = random.randint(-MAX_SHIFT, MAX_SHIFT)
        shifted = shift_matrix(mat, dx, dy)

        # 2. 輕微縮放
        scale = random.uniform(MIN_SCALE, MAX_SCALE)
        scaled = scale_matrix(shifted, scale)

        # 3. 智能噪點 (改進版本)
        noisy = add_smart_noise(scaled, BASE_NOISE, EDGE_NOISE)
        augmented_data.append(noisy)

        # 4. 可選旋轉 (30%機率，降低頻率)
        if random.random() < 0.3:
            rotated = rotate_matrix(noisy, random.randint(0, 1))
            augmented_data.append(rotated)

        if len(augmented_data) >= TARGET_COUNT:
            break

# 按照指定格式輸出1000筆資料
for i, mat in enumerate(augmented_data):
    print(" [")
    for row in mat:
        print("  " + str(row) + ",")
    print(" ]," + ("" if i == len(augmented_data)-1 else "\n"))

print(f"\n總共生成矩陣筆數: {len(augmented_data)}")

 [
  [0, 0, 0, 1, 1, 1, 1, 1],
  [0, 0, 0, 1, 0, 0, 0, 1],
  [0, 0, 0, 0, 1, 0, 1, 1],
  [0, 0, 0, 1, 1, 1, 1, 1],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 1, 1, 1, 0, 0],
 ],

 [
  [0, 0, 0, 1, 0, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 1, 1, 1, 1, 1, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
 ],

 [
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0],
  [1, 0, 0, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0],
 ],

 [
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0],
  [1, 0, 0, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0],
 ],

 [
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0